In [231]:
from src.jdv_conn import query_JDV
import pandas as pd
import random
pd.set_option('display.max_rows', None)
pd.set_option("display.colheader_justify","right")

# Load kcs and cases from the database

In [147]:
import json
kcs_rule_dict = json.load(open( "../data/kcs_rule_dict.json"))

In [150]:
ten_keys = []
for i in range(0,100):
    ten_keys.append(random.choice(list(kcs_rule_dict)))
kcs_string= ','.join(ten_keys)
kcs_string = ','.join(valid_kcs)

In [167]:
sql ="""
        SELECT kcs.resource_display_id__c, c.casenumber, c.subject, c.description
        FROM (SELECT  id, casenumber, subject, description
              FROM stg_gss_case
              WHERE stg_curr_flg = true
                  AND case_language__c='en'
                  AND resolution1__c LIKE '%Answer Provided%'
                  AND isdeleted = false
                  AND ownerid != '00GA0000000XxxNMAS') c
            INNER JOIN stg_gss_case_rsrc_rltnshp kcs ON kcs.case__c = c.id
        WHERE stg_curr_flg = true
            AND isdeleted = false
            AND type__c = 'Link'  
            AND kcs.resource_display_id__c IN (""" + kcs_string + """)
        ORDER BY kcs.resource_display_id__c
"""

In [168]:
cases = query_JDV(queryString=sql)

In [170]:
df = cases

In [66]:
df.head(10).style.set_properties(**{'text-align': 'left'})


In [156]:
count_df = df.groupby(['resource_display_id__c'])['casenumber'].agg(['count'])

In [159]:
count_df = count_df.reset_index()

In [160]:
valid_kcs = count_df[(count_df['count'] > 3) & (count_df['count'] < 30)]['resource_display_id__c'].tolist()

# Create case pair for BERT Model

In [171]:
# 遍历df
rule_tag_dict = {}
case_kcs_dict = {}
train_df = None
test_df = None
for key in ten_keys:
    temp_df = df[df['resource_display_id__c'] == key]
    temp_list = temp_df['casenumber'].tolist()
    for c in temp_list:
        case_kcs_dict[c] = key
    case_count = temp_df.shape[0]
    train_count = int(case_count * (4/5))
    temp_train_df = temp_df[:train_count-1]
    temp_test_df = temp_df[train_count:]
    if train_df is None:
        train_df = temp_train_df
    else:
        train_df = pd.concat([temp_train_df, train_df])
    if test_df is None:
        test_df = temp_test_df
    else:
        test_df = pd.concat([temp_test_df, test_df])        

In [172]:
train_df = train_df.sort_index(axis=0)
test_df = test_df.sort_index(axis=0)

In [173]:
test_set = []
for current_case in range(0, len(test_df)):
    for pair_case in range(0, len(train_df)):
        if test_df.iloc[current_case]['resource_display_id__c'] == train_df.iloc[pair_case]['resource_display_id__c']:
            test_set.append([test_df.iloc[current_case]['subject'], train_df.iloc[pair_case]['subject'], 1, test_df.iloc[current_case]['casenumber'], train_df.iloc[pair_case]['casenumber'], test_df.iloc[current_case]['resource_display_id__c'], train_df.iloc[pair_case]['resource_display_id__c']])
        else:
            test_set.append([test_df.iloc[current_case]['subject'], train_df.iloc[pair_case]['subject'], 0, test_df.iloc[current_case]['casenumber'], train_df.iloc[pair_case]['casenumber'], test_df.iloc[current_case]['resource_display_id__c'], train_df.iloc[pair_case]['resource_display_id__c']])


In [174]:
test_df = test_df.sort_index(axis=0)

In [175]:
case_pair_df = pd.DataFrame(test_set, columns=['casea','caseb','label','casea_id', 'caseb_id', 'casea_kcs', 'caseb_kcs'])

In [176]:
case_pair_df.head(10)

,casea,caseb,label,casea_id,caseb_id,casea_kcs,caseb_kcs
0,Unable to start firewalld use systemctl command,os boot fails with failed to start firewalld a...,1,02313355,01966384,1122173,1122173
1,Unable to start firewalld use systemctl command,Instance Startup at boot time issue with nssld...,1,02313355,01566050,1122173,1122173
2,Unable to start firewalld use systemctl command,"Firewalld service in failed state, unable to s...",1,02313355,01760155,1122173,1122173
3,Unable to start firewalld use systemctl command,Unable to migrate manifest,0,02313355,02007095,1122173,1349373
4,Unable to start firewalld use systemctl command,Updating Satellite manifest fails,0,02313355,02457529,1122173,1349373
5,Unable to start firewalld use systemctl command,"Unable to sync repositories, manifest seems ko",0,02313355,01857910,1122173,1349373
6,Unable to start firewalld use systemctl command,Unable to import Manifest,0,02313355,02412785,1122173,1349373
7,Unable to start firewalld use systemctl command,We need help moving manifest from one organiza...,0,02313355,02284031,1122173,1349373
8,Unable to start firewalld use systemctl command,Unable to refresh manifests,0,02313355,02289384,1122173,1349373
9,Unable to start firewalld use systemctl command,Create new subscriptions manifest,0,02313355,02055349,1122173,1349373


In [177]:
case_pair_df['guid'] = case_pair_df.index.values.tolist()

In [178]:
case_pair_df.to_pickle('../data/2rule_case_pair.pkl')

In [179]:
len(case_pair_df)

31616

# Running the recognizer to predict the test set cases in the backend

# Evaluate the match result

In [180]:
validate_df = pd.read_pickle('../data/2rule_case_pair.pkl')

In [181]:
test_result_df = pd.read_csv('../data/2/test_results.tsv', sep='\t')

In [182]:
test_result_df.head(3)

,non-pair,pair
0,0.157252,0.842748
1,0.834493,0.165507
2,0.197410,0.802590


In [183]:
validate_df['predict'] = test_result_df['pair'].to_list()

In [184]:
validate_df['predict_label'] = validate_df.apply(lambda x: 1 if x['predict'] >= 0.5 else 0,  axis=1) 

# Accuracy of the recognizer

In [185]:
total = 0
total_true = 0
for index,row in validate_df.iterrows():
    if row['label'] == row['predict_label']:
        total_true += 1
    total += 1
total_true/total

0.9419597672064778

In [230]:
validate_df.head(10)

,casea,caseb,label,casea_id,caseb_id,casea_kcs,caseb_kcs,guid,predict,predict_label
0,Unable to start firewalld use systemctl command,os boot fails with failed to start firewalld a...,1,02313355,01966384,1122173,1122173,0,0.842748,1
1,Unable to start firewalld use systemctl command,Instance Startup at boot time issue with nssld...,1,02313355,01566050,1122173,1122173,1,0.165507,0
2,Unable to start firewalld use systemctl command,"Firewalld service in failed state, unable to s...",1,02313355,01760155,1122173,1122173,2,0.802590,1
3,Unable to start firewalld use systemctl command,Unable to migrate manifest,0,02313355,02007095,1122173,1349373,3,0.164483,0
4,Unable to start firewalld use systemctl command,Updating Satellite manifest fails,0,02313355,02457529,1122173,1349373,4,0.162447,0
5,Unable to start firewalld use systemctl command,"Unable to sync repositories, manifest seems ko",0,02313355,01857910,1122173,1349373,5,0.163939,0
6,Unable to start firewalld use systemctl command,Unable to import Manifest,0,02313355,02412785,1122173,1349373,6,0.164535,0
7,Unable to start firewalld use systemctl command,We need help moving manifest from one organiza...,0,02313355,02284031,1122173,1349373,7,0.165707,0
8,Unable to start firewalld use systemctl command,Unable to refresh manifests,0,02313355,02289384,1122173,1349373,8,0.164055,0
9,Unable to start firewalld use systemctl command,Create new subscriptions manifest,0,02313355,02055349,1122173,1349373,9,0.162977,0


# Calculate the Similarity between 'new cases' and hit rules

In [187]:
mean_df = validate_df.groupby(['casea_id', 'casea_kcs', 'caseb_kcs'])['predict'].agg(['mean', 'max'])

In [188]:
mean_df = mean_df.loc[mean_df.groupby(['casea_id', 'casea_kcs'])['mean'].idxmax()].reset_index()

In [190]:
mean_df.head(10)

,casea_id,casea_kcs,caseb_kcs,mean,max
0,01629695,8721,21590,0.455332,0.899757
1,01721456,29537,29537,0.483475,0.899758
2,01778798,2127091,3266571,0.324792,0.800488
3,01799794,29537,3353861,0.674930,0.901040
4,01810358,722773,3353861,0.385770,0.534340
5,01830062,2039613,3353861,0.504652,0.631598
6,01850116,2127091,3353861,0.746530,0.902042
7,01852179,41042,3263161,0.221164,0.853800
8,01855477,41042,3353861,0.804099,0.902121
9,01863186,2124351,3734981,0.267471,0.860136


In [209]:
filter_df = validate_df.groupby(['casea_id', 'casea_kcs', 'caseb_kcs','casea'])['predict'].agg(['mean']).reset_index()

In [210]:
filter_df['rule'] = filter_df.apply(lambda x: kcs_rule_dict[x['caseb_kcs']], axis=1)

In [211]:
filter_df[filter_df['mean'] > 0.2]

,casea_id,casea_kcs,caseb_kcs,casea,mean,rule
7,01629695,8721,21590,Packet loss frequently,0.455332,networking/scp_get_disconnected_on_rhel6.py
35,01629695,8721,8721,Packet loss frequently,0.445312,networking/nf_conntrack_table_full.py
47,01721456,29537,29537,Multipathd errors for ASM devices PRCUSTDB01 P...,0.483475,storage/storage_multipath_asm_error_in_message...
77,01778798,2127091,2124351,I/O Errors on Virtual Server,0.240982,sysmgmt/satellite/sat_goferd_frequent_disconne...
78,01778798,2127091,2127091,I/O Errors on Virtual Server,0.206810,storage/scsi_device_reset.py
92,01778798,2127091,3266571,I/O Errors on Virtual Server,0.324792,shift/ocp_tls_handshake_error.py
96,01778798,2127091,3508421,I/O Errors on Virtual Server,0.318596,shift/ocp_clean_docker_storage.py
110,01799794,29537,157393,Server got rebooted due to Kernel panic,0.429045,filesystem/ext4_fs_error_double_free_of_inode.py
111,01799794,29537,2039613,Server got rebooted due to Kernel panic,0.503377,kernel/kernel_panic_at_hip_access_authorize.py
114,01799794,29537,2127091,Server got rebooted due to Kernel panic,0.307457,storage/scsi_device_reset.py


In [224]:
filter_df[filter_df['casea_kcs'] == filter_df['caseb_kcs']]

,casea_id,casea_kcs,caseb_kcs,casea,mean,rule
35,01629695,8721,8721,Packet loss frequently,0.445312,networking/nf_conntrack_table_full.py
47,01721456,29537,29537,Multipathd errors for ASM devices PRCUSTDB01 P...,0.483475,storage/storage_multipath_asm_error_in_message...
78,01778798,2127091,2127091,I/O Errors on Virtual Server,0.206810,storage/scsi_device_reset.py
119,01799794,29537,29537,Server got rebooted due to Kernel panic,0.227350,storage/storage_multipath_asm_error_in_message...
178,01810358,722773,722773,RH 6.6 and CPU E5-2667 v4 @ 3.20GHz,0.307047,kernel/unsupported_intel_cpu_model_log.py
183,01830062,2039613,2039613,Server crashed and need to determine the issue...,0.270338,kernel/kernel_panic_at_hip_access_authorize.py
222,01850116,2127091,2127091,server went in hung state,0.308609,storage/scsi_device_reset.py
283,01852179,41042,41042,系统无法登入系统及安装在RHEL上的数据库,0.155809,storage/storage_unable_to_read_sfp_data.py
319,01855477,41042,41042,Servers crash during storage paths recovery,0.286675,storage/storage_unable_to_read_sfp_data.py
329,01863186,2124351,2124351,katello-agent導入後のメッセージについて（20261414）,0.167353,sysmgmt/satellite/sat_goferd_frequent_disconne...


In [227]:
filter_df[(filter_df['casea_kcs'] == filter_df['caseb_kcs']) & (filter_df['mean'] <= 0.2)].style.set_properties(**{'text-align': 'left'})


,casea_id,casea_kcs,caseb_kcs,casea,mean,rule
283,01852179,41042,41042,系统无法登入系统及安装在RHEL上的数据库,0.155809,storage/storage_unable_to_read_sfp_data.py
329,01863186,2124351,2124351,katello-agent導入後のメッセージについて（20261414）,0.167353,sysmgmt/satellite/sat_goferd_frequent_disconnected.py
375,01883671,3059761,3059761,MQ objects damaged on pushing loads of messages,0.160769,shift/ocp_iptables_errors_resources_temp_unavailable.py
499,01905430,41042,41042,System rebooted unexpectedly. Error logs shows kernel crash. Need root cause for the crash.,0.167999,storage/storage_unable_to_read_sfp_data.py
506,01918037,157393,157393,Error while migration of vms in KVM,0.165076,filesystem/ext4_fs_error_double_free_of_inode.py
723,01980429,2039613,2039613,"IPv6: Loaded, but administratively disabled, reboot required to enable",0.173377,kernel/kernel_panic_at_hip_access_authorize.py
768,01999013,2994531,2994531,multipathd: 8:16: mark as failed,0.167207,rhev/rhev_sense_key_error.py
909,02038797,2750581,2750581,Red Hat Enterprise Linux 6/7 Server - Oracle Java RPMs x86_64 has no packages or errata,0.166819,sysmgmt/satellite/sat6_import_manifest_fails.py
951,02040180,3059761,3059761,"In development cluster, appnode running very high on load average.",0.173056,shift/ocp_iptables_errors_resources_temp_unavailable.py
1130,02066982,3034841,3034841,[BUG] Stuck tasks: Destroy Content Host,0.168329,sysmgmt/satellite/sat6_pulp_logs_node_not_found.py


In [218]:
result_df = filter_df[filter_df['mean'] > 0.2]

In [223]:
len(filter_df.groupby(['casea_id']).groups)

102

In [221]:
len(result_df[result_df['casea_kcs'] == result_df['caseb_kcs']])

65

# The recall value of finding the correct KCS

In [225]:
65/102

0.6372549019607843

In [220]:
result_df[result_df['casea_kcs'] == result_df['caseb_kcs']]

,casea_id,casea_kcs,caseb_kcs,casea,mean,rule
35,01629695,8721,8721,Packet loss frequently,0.445312,networking/nf_conntrack_table_full.py
47,01721456,29537,29537,Multipathd errors for ASM devices PRCUSTDB01 P...,0.483475,storage/storage_multipath_asm_error_in_message...
78,01778798,2127091,2127091,I/O Errors on Virtual Server,0.206810,storage/scsi_device_reset.py
119,01799794,29537,29537,Server got rebooted due to Kernel panic,0.227350,storage/storage_multipath_asm_error_in_message...
178,01810358,722773,722773,RH 6.6 and CPU E5-2667 v4 @ 3.20GHz,0.307047,kernel/unsupported_intel_cpu_model_log.py
183,01830062,2039613,2039613,Server crashed and need to determine the issue...,0.270338,kernel/kernel_panic_at_hip_access_authorize.py
222,01850116,2127091,2127091,server went in hung state,0.308609,storage/scsi_device_reset.py
319,01855477,41042,41042,Servers crash during storage paths recovery,0.286675,storage/storage_unable_to_read_sfp_data.py
404,01897501,2380591,2380591,/var/log/massage file gradually increases,0.474601,service/audispd_log_messages.py
438,01905186,2127091,2127091,iowait issue HYDFFMPRDAPP9T3,0.314618,storage/scsi_device_reset.py
